### Installing all the necessary libraries

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 104.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

### Then importing all the needed libraries

In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, BitsAndBytesConfig, Trainer, pipeline
from peft import LoraConfig
from datasets import Dataset
from datasets import load_dataset
from langchain.prompts.prompt import PromptTemplate

from trl import SFTTrainer
from peft import PeftModel

import pandas as pd

In [ ]:
model_name="meta-llama/Llama-2-7b-hf"

### loading  the model in 4-bit precision

In [ ]:
compute_dtype = getattr(torch, "float16")

bnb_config = BitsAndBytesConfig(
load_in_4bit=True,
bnb_4bit_use_double_quant=True,
bnb_4bit_quant_type="nf4",
bnb_4bit_compute_dtype=torch.bfloat16
)
#tokenizer from any model can be taken
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

### We need to be careful about the data we use to finetune the model , so the below code is for when we have the data in CSV form and we want to format the data into the prompt template of llama2

In [ ]:
df_data = pd.read_excel("/content/Gen AI Questions nish.xlsx")

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["category", "question", "SQL_Queries"], template="<s>[INST]<<SYS>>You are an agent designed to interact with a SQL database.Given an input question, create a syntactically correct teradatasql query to run. This is a {category} question. </SYS>{question}[/INST]{sql_query}</s>"
)

prompt_data = []
for i, row in df_data.iterrows():
  prompt_data.append(prompt_template.format(category=row.category, question=row.question, sql_query=row.SQL_Queries))

dataset = Dataset.from_dict({"inputs": prompt_data})

In [ ]:
print(dataset[2].get('inputs'))

<s>[INST]<<SYS>>You are an agent designed to interact with a SQL database.Given an input question, create a syntactically correct teradatasql query to run. This is a Bank Balances question. </SYS>What is my currency exposure based on my bank balances in all my bank accounts?[/INST]SELECT currency_code, SUM((balance->>'available')::numeric) AS total_balance FROM bank_accounts b GROUP BY currency_code ORDER BY total_balance DESC</s>


In [ ]:
dataset.to_json("./data/finetune_data.json")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

27362

## Loading the dataset from huggingface

In [ ]:
data_name = "mlabonne/guanaco-llama2-1k"
dataset = load_dataset(data_name, split="train")

### from here on, the training arguments settings and training starts:-

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
peft_params = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    logging_steps=1,
    learning_rate=2e-4,
    fp16=True
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.320700
2,1.399100
3,1.418800
4,1.383600
5,1.378200
6,1.561100
7,1.592300
8,1.300300
9,1.353100
10,1.904700


Step,Training Loss
1,1.320700
2,1.399100
3,1.418800
4,1.383600
5,1.378200
6,1.561100
7,1.592300
8,1.300300
9,1.353100
10,1.904700


In [ ]:
new_model="llama2-sql-3-sft"

In [ ]:
new_model = "./drive/MyDrive/finetune_model/llama3_sql-sft"

trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

('./drive/MyDrive/finetune_model/llama2_sql-sft/tokenizer_config.json',
 './drive/MyDrive/finetune_model/llama2_sql-sft/special_tokens_map.json',
 './drive/MyDrive/finetune_model/llama2_sql-sft/tokenizer.json')

## The above cell saves the LoRA adapters onto the folder along with the tokenizer

### Now as we need to merge the adapters with the base model

In [ ]:
model_name = "meta-llama/Meta-Llama-3-8B"
new_model = "/content/finetuned_model"

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# 1. Load based model
base_model = AutoModelForCausalLM.from_pretrained(model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

In [ ]:
# 2. Load new fine-tuned model. Then merge this two model
finetuned_model = PeftModel.from_pretrained(base_model, new_model)
merged_model = finetuned_model.merge_and_unload()

In [ ]:
from huggingface_hub import login
login()

#### uploading merged model into huggingface hub


In [ ]:
merged_model.push_to_hub("llama3-sql-sft-merged")